In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
import sklearn.metrics as metrics
from nltk.stem import PorterStemmer
import spacy

In [2]:
df = pd.read_csv('../../Data/master_df.csv')

this is the baseline accuracy. I need to beat ~50.8% accuracy

In [4]:
df['is_conspiracy'].mean()

0.49197288619336427

the `'X'` column has 11996 rows where it is 0. I'm not sure what part of speech it is and will look into it later

In [6]:
just_pos = df.drop(columns=['text', 'is_conspiracy', 'X'])

y = df['is_conspiracy']
X = just_pos
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify = y,
                                                    random_state = 42)

In [7]:
# data is stratified. shuffle before cross validation
folds = StratifiedKFold(shuffle=True, random_state = 42)

In [145]:
# https://www.geeksforgeeks.org/python-format-function/
cv_score = cross_val_score(lr, X, y, cv = folds)
print('{} \u00B1 {}'.format(cv_score.mean(), cv_score.std()))
print(cv_score)

0.7556179314446256 ± 0.009627959874752472
[0.76189061 0.76575505 0.7568371  0.75587273 0.73773417]


In [150]:
coef_df = pd.DataFrame(lr.coef_,
                       columns=X_train.columns)

coef_df.T.sort_values(by=0, ascending=False)

,0
PROPN,25.518761
PRON,20.986247
ADJ,14.860595
SCONJ,13.527757
CCONJ,9.925618
SPACE,8.694940
ADP,4.978184
NUM,0.000000
PART,0.000000
PUNCT,0.000000


**Interpretation:** This model has a ~25% increase in accuracy as compared to the baseline. This is great considering the model is only trained on parts of speech. Conspiracy posts tend to use proper nouns, pronouns, and adjectives more. Regular posts tend to use nouns, syms?, and whatever the hell X is.

In [10]:
X.shape, df.shape

((16818, 17), (16818, 21))

In [11]:
pipe = Pipeline([

    ('model', LogisticRegression(max_iter=1000)),
])

In [12]:
params = {
    'model__penalty': ['l1'],
    'model__C': [1.0],
    'model__solver': ['liblinear']
    
}

gs = GridSearchCV(pipe, param_grid=params)

gs.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('model',
                                        LogisticRegression(max_iter=1000))]),
             param_grid={'model__C': [1.0], 'model__penalty': ['l1'],
                         'model__solver': ['liblinear']})

In [13]:
gs.best_params_

{'model__C': 1.0, 'model__penalty': 'l1', 'model__solver': 'liblinear'}

In [14]:
gs.best_score_

0.7272662009119589

In [15]:
gs.score(X_test, y_test), gs.score(X_train, y_train)

(0.7414982164090369, 0.7284547688892412)

In [16]:
lr = LogisticRegression(penalty = 'l1',
                        solver='liblinear', max_iter=1000)

In [17]:
lr.fit(X_train, y_train)

LogisticRegression(max_iter=1000, penalty='l1', solver='liblinear')

In [18]:
lr.score(X_test, y_test)

0.7414982164090369

In [19]:
coef_df = pd.DataFrame(lr.coef_,
                       columns=X_train.columns)

In [105]:
np.exp(coef_df['SYM'][0])

6.687191289769441e-08

In [92]:
np.exp(coef_df['SYM'][0])

1.0718006016551185